CTGAN Model
===========

In this guide we will go through a series of steps that will let you
discover functionalities of the `CTGAN` model, including how to:

-   Create an instance of `CTGAN`.
-   Fit the instance to your data.
-   Generate synthetic versions of your data.
-   Use `CTGAN` to anonymize PII information.
-   Customize the data transformations to improve the learning process.
-   Specify hyperparameters to improve the output quality.

What is CTGAN?
--------------

The `sdv.tabular.CTGAN` model is based on the GAN-based Deep Learning
data synthesizer which was presented at the NeurIPS 2020 conference by
the paper titled [Modeling Tabular data using Conditional
GAN](https://arxiv.org/abs/1907.00503).

Let\'s now discover how to learn a dataset and later on generate
synthetic data with the same format and statistical properties by using
the `CTGAN` class from SDV.

Quick Usage
-----------

We will start by loading one of our demo datasets, the
`student_placements`, which contains information about MBA students that
applied for placements during the year 2020.

<div class="alert alert-warning">

**Warning**

In order to follow this guide you need to have `ctgan` installed on your
system. If you have not done it yet, please install `ctgan` now by
executing the command `pip install sdv` in a terminal.

</div>

In [2]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


As you can see, this table contains information about students which
includes, among other things:

-   Their id and gender
-   Their grades and specializations
-   Their work experience
-   The salary that they where offered
-   The duration and dates of their placement

You will notice that there is data with the following characteristics:

-   There are float, integer, boolean, categorical and datetime values.
-   There are some variables that have missing data. In particular, all
    the data related to the placement details is missing in the rows
    where the student was not placed.

Let us use `CTGAN` to learn this data and then sample synthetic data
about new students to see how well de model captures the characteristics
indicated above. In order to do this you will need to:

-   Import the `sdv.tabular.CTGAN` class and create an instance of it.
-   Call its `fit` method passing our table.
-   Call its `sample` method indicating the number of synthetic rows
    that you want to generate.

In [3]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

<div class="alert alert-info">

**Note**

Notice that the model `fitting` process took care of transforming the
different fields using the appropriate [Reversible Data
Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has a
format that the underlying CTGANSynthesizer class can handle.

</div>

### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic
data by calling the `sample` method from your model passing the number
of rows that we want to generate.

In [4]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted
on, but filled with new data which resembles the original one.

In [5]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17273,M,76.671040,63.534093,Science,54.799948,Sci&Tech,False,0,53.508584,Mkt&Fin,68.920151,NaN,False,NaT,NaT,NaN
1,17432,F,78.910800,56.848729,Science,78.901273,Sci&Tech,False,0,70.839514,Mkt&Fin,58.653935,36274.703622,True,2020-03-04,2020-08-01,3.0
2,17514,M,87.659415,43.118437,Commerce,89.887399,Sci&Tech,False,0,69.633209,Mkt&Fin,47.657597,NaN,False,2020-02-06,NaT,NaN
3,17274,M,92.297661,83.910693,Science,62.895575,Sci&Tech,False,0,98.691861,Mkt&HR,55.287540,30279.902702,False,2020-02-16,NaT,3.0
4,17369,F,69.314481,58.445286,Commerce,62.848358,Comm&Mgmt,True,0,91.765925,Mkt&Fin,57.300165,NaN,False,NaT,NaT,12.0


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples`
in the `model.sample(<num_rows>)`. To test, try `model.sample(10000)`.
Note that the original table only had \~200 rows.

</div>

### Save and Load the model

In many scenarios it will be convenient to generate synthetic versions
of your data directly in systems that do not have access to the original
data source. For example, if you may want to generate testing data on
the fly inside a testing environment that does not have access to your
production database. In these scenarios, fitting the model with real
data every time that you need to generate new data is feasible, so you
will need to fit a model in your production environment, save the fitted
model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let\'s see how this process works.

#### Save and share the model

Once you have fitted the model, all you need to do is call its `save`
method passing the name of the file in which you want to save the model.
Note that the extension of the filename is not relevant, but we will be
using the `.pkl` extension to highlight that the serialization protocol
used is [pickle](https://docs.python.org/3/library/pickle.html).

In [6]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same
directory in which you are running SDV.

<div class="alert alert-info">

**Important**

If you inspect the generated file you will notice that its size is much
smaller than the size of the data that you used to generate it. This is
because the serialized model contains **no information about the
original data**, other than the parameters it needs to generate
synthetic versions of it. This means that you can safely share this
`my_model.pkl` file without the risc of disclosing any of your real
data!

</div>

#### Load the model and generate new data

The file you just generated can be send over to the system where the
synthetic data will be generated. Once it is there, you can load it
using the `CTGAN.load` method, and then you are ready to sample new data
from the loaded instance:

In [7]:
loaded = CTGAN.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">

**Warning**

Notice that the system where the model is loaded needs to also have
`sdv` and `ctgan` installed, otherwise it will not be able to load the
model and use it.

</div>

### Specifying the Primary Key of the table

One of the first things that you may have noticed when looking that demo
data is that there is a `student_id` column which acts as the primary
key of the table, and which is supposed to have unique values. Indeed,
if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [8]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe
that there are some values that appear more than once:

In [9]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
26,17375,F,73.067268,61.448859,Commerce,65.075293,Sci&Tech,False,0,84.968623,Mkt&HR,52.774263,26422.335703,True,2020-01-24,NaT,6.0
31,17375,F,82.008072,64.951992,Science,64.661266,Comm&Mgmt,True,0,65.465880,Mkt&HR,65.245901,NaN,False,2020-03-25,NaT,NaN
51,17375,M,73.796830,49.154957,Commerce,85.955449,Comm&Mgmt,False,0,70.486734,Mkt&HR,53.436912,37902.795261,True,2020-01-29,2020-07-18,3.0
74,17375,F,82.725955,59.629648,Commerce,65.017346,Comm&Mgmt,False,0,57.605865,Mkt&HR,61.358357,22619.396259,True,2020-02-04,2020-05-04,3.0


This happens because the model was not notified at any point about the
fact that the `student_id` had to be unique, so when it generates new
data it will provoke collisions sooner or later. In order to solve this,
we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [10]:
model = CTGAN(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,F,46.202478,71.944795,Commerce,58.749559,Sci&Tech,True,0,46.649107,Mkt&Fin,61.348999,19792.430278,False,NaT,2020-12-02,NaN
1,1,F,44.007657,56.152927,Science,71.279806,Sci&Tech,False,0,49.314496,Mkt&HR,64.742902,NaN,False,2020-02-17,NaT,NaN
2,2,M,60.113445,86.070652,Commerce,80.518311,Comm&Mgmt,False,0,46.106460,Mkt&HR,69.756731,21877.657432,False,NaT,2020-09-22,6.0
3,3,F,78.893156,49.938424,Commerce,80.548343,Comm&Mgmt,False,0,87.252770,Mkt&Fin,72.869287,NaN,True,2020-02-11,2020-09-02,12.0
4,4,M,65.008961,54.406701,Commerce,61.073019,Comm&Mgmt,True,0,43.785671,Mkt&Fin,66.662136,22090.623519,True,2020-03-09,2020-07-04,6.0


As a result, the model will learn that this column must be unique and
generate a unique sequence of values for the column:

In [11]:
new_data.student_id.value_counts().max()

1

### Anonymizing Personally Identifiable Information (PII)

There will be many cases where the data will contain Personally
Identifiable Information which we cannot disclose. In these cases, we
will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but
does not contain any of the original values.

Let\'s load a new dataset that contains a PII field, the
`student_placements_pii` demo, and try to generate synthetic versions of
it that do not contain any of the PII fields.

<div class="alert alert-info">

**Note**

The `student_placements_pii` dataset is a modified version of the
`student_placements` dataset with one new field, `address`, which
contains PII information about the students. Notice that this additional
`address` field has been simulated and does not correspond to data from
the real users.

</div>

In [12]:
data_pii = load_tabular_demo('student_placements_pii')
data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,"70304 Baker Turnpike\nEricborough, MS 15086",M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,"805 Herrera Avenue Apt. 134\nMaryview, NJ 36510",M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,"3702 Bradley Island\nNorth Victor, FL 12268",M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,Unit 0879 Box 3878\nDPO AP 42663,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,"96493 Kelly Canyon Apt. 145\nEast Steven, NC 3...",M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


If we use our tabular model on this new data we will see how the
synthetic data that it generates discloses the addresses from the real
students:

In [13]:
model = CTGAN(
    primary_key='student_id',
)
model.fit(data_pii)
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"6994 White Falls\nLake Cynthiaborough, CT 15835",F,76.679200,55.281644,Science,54.567875,Others,True,0,68.668355,Mkt&Fin,52.553022,21435.640261,True,NaT,2020-09-26,NaN
1,1,USNV Baldwin\nFPO AE 29267,M,79.870296,102.384728,Commerce,46.569330,Sci&Tech,True,1,65.230831,Mkt&Fin,47.557286,31406.514934,True,2020-08-28,2020-10-06,3.0
2,2,"PSC 1354, Box 1414\nAPO AA 25325",F,86.966156,60.777417,Commerce,56.550707,Comm&Mgmt,False,0,72.254769,Mkt&HR,54.058065,31507.054218,True,2020-02-06,2020-09-29,3.0
3,3,"003 Donna Trace Apt. 736\nNorth Johnburgh, NH ...",M,76.653846,76.912513,Science,57.327801,Sci&Tech,False,2,67.689928,Mkt&Fin,66.190216,NaN,True,2020-03-27,2020-11-23,NaN
4,4,"90070 Gregory Center\nLynchport, MA 04844",M,64.740630,69.593230,Commerce,57.474791,Comm&Mgmt,True,3,48.417371,Mkt&HR,53.911995,NaN,True,NaT,2020-12-10,12.0


More specifically, we can see how all the addresses that have been
generated actually come from the original dataset:

In [14]:
new_data_pii.address.isin(data_pii.address).sum()

200

In order to solve this, we can pass an additional argument
`anonymize_fields` to our model when we create the instance. This
`anonymize_fields` argument will need to be a dictionary that contains:

-   The name of the field that we want to anonymize.
-   The category of the field that we want to use when we generate fake
    values for it.

The list complete list of possible categories can be seen in the [Faker
Providers](https://faker.readthedocs.io/en/master/providers.html) page,
and it contains a huge list of concepts such as:

-   name
-   address
-   country
-   city
-   ssn
-   credit_card_number
-   credit_card_expire
-   credit_card_security_code
-   email
-   telephone
-   \...

In this case, since the field is an e-mail address, we will pass a
dictionary indicating the category `address`

In [15]:
model = CTGAN(
    primary_key='student_id',
    anonymize_fields={
        'address': 'address'
    }
)
model.fit(data_pii)

As a result, we can see how the real `address` values have been replaced
by other fake addresses:

In [16]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"5494 Watkins Row Apt. 585\nWest Michael, MD 37948",M,44.545314,49.462338,Science,70.440045,Sci&Tech,True,0,98.474596,Mkt&HR,64.500169,NaN,True,NaT,2020-11-26,NaN
1,1,"86102 Robert Orchard\nCookmouth, OH 99064",F,63.156398,74.163211,Commerce,46.109824,Comm&Mgmt,False,1,54.392178,Mkt&Fin,62.250115,31071.925966,True,2020-07-16,2020-04-18,NaN
2,2,"16230 Jason Via\nNorth Ethanborough, ID 01624",F,43.267048,65.925501,Science,66.288700,Sci&Tech,False,0,59.138077,Mkt&HR,68.994737,30877.712514,False,2020-01-30,NaT,NaN
3,3,"5307 Ricky Plaza Suite 837\nEdwardsside, WV 17549",M,47.372743,58.074479,Commerce,58.854752,Comm&Mgmt,False,0,57.331987,Mkt&Fin,74.887428,31297.987387,False,NaT,NaT,3.0
4,4,"210 Chad Place\nLake Christina, AZ 79936",M,46.010370,54.254208,Commerce,48.356893,Comm&Mgmt,False,1,67.259702,Mkt&HR,78.232336,45216.771941,True,2020-03-26,2021-01-20,12.0


Which means that none of the original addresses can be found in the
sampled data:

In [17]:
data_pii.address.isin(new_data_pii.address).sum()

0

Advanced Usage
--------------

Now that we have discovered the basics, let\'s go over a few more
advanced usage examples and see the different arguments that we can pass
to our `CTGAN` Model in order to customize it to our needs.

### How to modify the CTGAN Hyperparameters?

A part from the common Tabular Model arguments, `CTGAN` has a number of
additional hyperparameters that control its learning behavior and can
impact on the performance of the model, both in terms of quality of the
generated data and computational time.

-   `epochs` and `batch_size`: these arguments control the number of
    iterations that the model will perform to optimize its parameters,
    as well as the number of samples used in each step. Its default
    values are `300` and `500` respectively, and `batch_size` needs to
    always be a value which is multiple of `10`. These hyperparameters
    have a very direct effect in time the training process lasts but
    also on the performance of the data, so for new datasets, you might
    want to start by setting a low value on both of them to see how long
    the training process takes on your data and later on increase the
    number to acceptable values in order to improve the performance.
-   `log_frequency`: Whether to use log frequency of categorical levels
    in conditional sampling. It defaults to `True`. This argument affects
    how the model processes the frequencies of the categorical values that
    are used to condition the rest of the values. In some cases, changing
    it to `False` could lead to better performance.
-   `embedding_dim` (int): Size of the random sample passed to the
    Generator. Defaults to 128.
-   `generator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Residuals. A Resiudal Layer will be created for each
    one of the values provided. Defaults to (256, 256).
-   `discriminator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Discriminator Layers. A Linear Layer will be created
    for each one of the values provided. Defaults to (256, 256).
-   `generator_lr` (float): Learning rate for the generator. Defaults to 2e-4.
-   `generator_decay` (float): Generator weight decay for the Adam Optimizer.
    Defaults to 1e-6.
-   `discriminator_lr` (float): Learning rate for the discriminator.
    Defaults to 2e-4.
-   `discriminator_decay` (float): Discriminator weight decay for the Adam
    Optimizer. Defaults to 1e-6.
-   `discriminator_steps` (int): Number of discriminator updates to do for
    each generator update. From the WGAN paper: https://arxiv.org/abs/1701.07875.
    WGAN paper default is 5. Default used is 1 to match original CTGAN
    implementation.
-   `verbose`: Whether to print fit progress on stdout. Defaults to
    `False`.

<div class="alert alert-warning">

**Warning**

Notice that the value that you set on the `batch_size` argument must
always be a multiple of `10`!

</div>

As an example, we will try to fit the `CTGAN` model slightly increasing
the number of epochs, reducing the `batch_size`, adding one additional
layer to the models involved and using a smaller wright decay.

Before we start, we will evaluate the quality of the previously
generated data using the `sdv.evaluation.evaluate` function

In [18]:
from sdv.evaluation import evaluate

evaluate(new_data, data)

0.3149921439127278

Afterwards, we create a new instance of the `CTGAN` model with the
hyperparameter values that we want to use

In [19]:
model = CTGAN(
    primary_key='student_id',
    epochs=500,
    batch_size=100,
    generator_dim=(256, 256, 256),
    discriminator_dim=(256, 256, 256)
)

And fit to our data.

In [21]:
model.fit(data)

Finally, we are ready to generate new data and evaluate the results.

In [22]:
new_data = model.sample(len(data))
evaluate(new_data, data)

0.2868219373546625

As we can see, in this case these modifications changed the obtained
results slightly, but they did neither introduce dramatic changes in the
performance.


### Conditional Sampling

As the name implies, conditional sampling allows us to sample from a conditional distribution using the CTGAN model, which means we can generate only values that satisfy certain conditions. These conditional values can be passed to the `conditions` parameter in the `sample` method either as a dataframe or a dictionary.

In case a dictionary is passed, the model will generate as many rows as requested, all of which will satisfy the specified conditions, such as `gender = M`.

In [23]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
model = CTGAN()
model.fit(data)
conditions = {
    "gender": 'M'
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17456,M,37.345473,52.436370,Science,70.255809,Sci&Tech,True,1,59.777470,Mkt&HR,52.199410,25814.981498,True,2020-01-04,2020-07-23,12.0
1,17444,M,61.603076,51.765820,Commerce,63.096284,Comm&Mgmt,False,0,77.672596,Mkt&Fin,46.190147,29395.811899,False,2020-01-10,NaT,NaN
2,17354,M,62.537037,55.921856,Commerce,63.645011,Others,False,0,85.597632,Mkt&Fin,63.030436,31074.308597,True,2020-06-18,2020-03-06,3.0
3,17380,M,49.264363,68.099206,Commerce,48.928634,Comm&Mgmt,True,0,81.702615,Mkt&HR,48.750736,22470.952387,True,2019-12-09,2020-09-29,3.0
4,17354,M,44.651287,27.076598,Commerce,54.640287,Comm&Mgmt,True,0,86.850449,Mkt&Fin,70.007107,NaN,True,2020-02-17,2020-07-19,12.0


If a dictionary of lists is passed instead, the model will generate one sample for each condition in the list, sorted in the same order. Since the model already knows how many samples to generate (one for each element in the list), passing it as a parameter is unnecessary. For example, if we want to generate three samples where `gender = M` and three samples with `gender = F`, we can do the following: 

In [24]:
conditions = {
    'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
}
model.sample(conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17434,M,41.695887,51.434056,Science,66.236310,Comm&Mgmt,True,0,77.166212,Mkt&HR,65.041786,30878.427037,True,2020-01-26,2020-09-05,NaN
1,17343,M,58.788508,74.637153,Commerce,62.201154,Comm&Mgmt,True,1,53.049858,Mkt&Fin,50.244320,NaN,True,NaT,2020-08-22,3.0
2,17311,M,81.719923,54.766715,Arts,68.750289,Sci&Tech,True,0,59.444630,Mkt&Fin,48.098813,29568.499598,True,NaT,2020-02-16,6.0
3,17296,F,90.916686,69.517114,Commerce,69.735516,Sci&Tech,False,0,94.894574,Mkt&HR,65.982235,NaN,True,2020-02-28,2020-06-09,NaN
4,17254,F,82.989872,66.383202,Commerce,57.336716,Comm&Mgmt,False,0,69.859504,Mkt&Fin,55.224178,NaN,True,2020-01-05,NaT,3.0
5,17206,F,66.930483,90.025683,Commerce,62.652187,Comm&Mgmt,True,0,61.043028,Mkt&HR,56.153820,NaN,True,2020-02-21,NaT,6.0


In either case, it's also possible to condition on multiple columns, such as `gender = M, work_experience = True, high_spec = Science`.

In [25]:
conditions = {
    'gender': 'M',
    'work_experience': True,
    'high_spec': 'Science'
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17507,M,49.476659,87.842702,Science,45.317095,Comm&Mgmt,True,0,60.806280,Mkt&Fin,63.352574,NaN,True,NaT,2020-04-09,3.0
1,17391,M,34.856014,52.624840,Science,45.268480,Comm&Mgmt,True,0,75.392326,Mkt&Fin,56.626121,NaN,True,2020-01-31,NaT,NaN
2,17346,M,45.597870,66.614175,Science,53.164395,Comm&Mgmt,True,1,99.332372,Mkt&HR,48.028726,NaN,True,NaT,NaT,3.0
3,17363,M,75.672783,70.977362,Science,70.466569,Others,True,0,77.342025,Mkt&Fin,52.208372,25773.867876,True,NaT,2020-06-07,12.0
4,17245,M,46.074062,34.840737,Science,60.502961,Comm&Mgmt,True,1,54.607479,Mkt&Fin,53.050502,32169.085481,False,NaT,2020-06-30,3.0


`conditions` can also be passed as a dataframe, and the behaviour will be the same as for a dictionary of lists.

In [26]:
conditions = pd.DataFrame({
    'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
    'work_experience': [True, True, True, True, True, True]
})
model.sample(conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17409,M,39.502010,70.518560,Science,52.778083,Comm&Mgmt,True,1,68.250398,Mkt&Fin,48.823499,28729.421099,True,2020-02-15,2020-11-28,NaN
1,17346,M,57.504880,59.196480,Science,42.621818,Comm&Mgmt,True,0,59.793413,Mkt&HR,51.872940,NaN,False,2019-12-27,2020-07-05,6.0
2,17279,M,61.337723,49.136300,Commerce,63.967621,Sci&Tech,True,0,68.688286,Mkt&HR,56.507923,28176.001301,True,NaT,NaT,NaN
3,17270,F,62.971748,42.439528,Commerce,59.488721,Comm&Mgmt,True,0,63.536729,Mkt&HR,49.460740,29755.356686,False,2019-12-18,2020-03-21,12.0
4,17354,F,52.592995,59.859225,Science,61.264885,Comm&Mgmt,True,0,70.657334,Mkt&Fin,68.908904,NaN,True,2020-01-06,2020-08-26,3.0
5,17430,F,64.019723,87.022308,Science,50.556498,Sci&Tech,True,0,64.885961,Mkt&Fin,55.393889,NaN,True,2020-01-19,2020-08-08,NaN


CTGAN also supports conditioning on continuous values, as long as the values are within the range of seen numbers. For example, if all the values of the dataset are within 0 and 1, CTGAN will not be able to set this value to 1000.

In [29]:
conditions = {
    'degree_perc': 70.0
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17388,M,73.928445,53.939157,Commerce,70.0,Comm&Mgmt,False,0,51.994453,Mkt&Fin,49.446954,30793.879025,True,2020-01-08,NaT,12.0
1,17335,F,47.175276,34.868506,Commerce,70.0,Sci&Tech,True,0,64.870702,Mkt&HR,67.747288,27454.307890,True,2019-12-14,2020-06-12,12.0
2,17390,M,53.011181,71.794440,Commerce,70.0,Sci&Tech,True,1,60.512838,Mkt&Fin,66.126284,32527.736354,True,2020-02-18,NaT,NaN
3,17384,F,44.131180,43.917175,Science,70.0,Sci&Tech,False,0,78.736036,Mkt&Fin,50.824366,NaN,True,2020-07-11,2020-03-11,NaN
4,17338,M,81.287353,37.823967,Commerce,70.0,Comm&Mgmt,False,0,91.623221,Mkt&Fin,51.962894,30267.089598,True,NaT,2020-07-06,NaN


<div class="alert alert-info">

**Note**
Currently, conditional sampling works through a rejection sampling process, where rows are sampled repeatedly until one that satisfies the conditions is found. In case you are running into a `Could not get enough valid rows within x trials` or simply wish to optimize the results, there are three parameters that can be fine-tuned: max_rows_multiplier, max_retries and float_rtol. More information about these parameters can be found in the API section.


</div>

### How do I specify constraints?

If you look closely at the data you may notice that some properties were
not completely captured by the model. For example, you may have seen
that sometimes the model produces an `experience_years` number greater
than `0` while also indicating that `work_experience` is `False`. These
type of properties are what we call `Constraints` and can also be
handled using `SDV`. For further details about them please visit the
[Handling Constraints](04_Handling_Constraints.ipynb) guide.

### Can I evaluate the Synthetic Data?

A very common question when someone starts using **SDV** to generate
synthetic data is: *\"How good is the data that I just generated?\"*

In order to answer this question, **SDV** has a collection of metrics
and tools that allow you to compare the *real* that you provided and the
*synthetic* data that you generated using **SDV** or any other tool.

You can read more about this in the [Evaluating Synthetic Data Generators](
05_Evaluating_Synthetic_Data_Generators.ipynb) guide.